<a href="https://colab.research.google.com/github/sgcortes/UrbanAccessibility/blob/main/01_GoogleStreetViewDownload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SCRIPT DESCARGA IMAGENES GOOGLE STREET VIEW

In [ ]:
from streetview import search_panoramas
from streetview.download import get_width_and_height_from_zoom, iter_tiles

from PIL import Image

import numpy as np

def autocrop(image, threshold=0):
	"""Crops any edges below or equal to threshold
	Crops blank image to 1x1.
	Returns cropped image.
	"""
	if len(image.shape) == 3:
		flatImage = np.max(image, 2)
	else:
		flatImage = image
	assert len(flatImage.shape) == 2

	rows = np.where(np.max(flatImage, 0) > threshold)[0]
	if rows.size:
		cols = np.where(np.max(flatImage, 1) > threshold)[0]
		image = image[cols[0]: cols[-1] + 1, rows[0]: rows[-1] + 1]
	else:
		image = image[:1, :1]

	return image

def props(cls):
	return [i for i in cls.__dict__.keys() if i[:1] != '_']


def get_panorama_tiles(pano_id: str, zoom: int = 5, multi_threaded: bool = False, outfile: str = "") -> Image.Image:
	"""
	Downloads a streetview panorama.
	Multi-threaded is a lot faster, but it's also a lot more likely to get you banned.
	"""

	tile_width = 512
	tile_height = 512

	total_width, total_height = get_width_and_height_from_zoom(zoom)
	panorama = Image.new("RGB", (total_width * tile_width, total_height * tile_height))

	it = 0

	for tile in iter_tiles(pano_id=pano_id, zoom=zoom, multi_threaded=multi_threaded):
		panorama.paste(im=tile.image, box=(tile.x * tile_width, tile.y * tile_height))
		tile.image.save("%s_%d.jpg" % (outfile, it))
		it+=1
		del tile

	return panorama

#panos = search_panoramas(lat=41.8982208, lon=12.4764804)
panos = search_panoramas(lat=43.3577835, lon=-5.8532249)

print("Found %d panoramas" % len(panos))

import csv

with open('output.csv','w') as f:
	writer = csv.writer(f)
	writer.writerow(props(panos[0]))
	writer.writerows([list(p.__dict__.values()) for p in panos])

#for i in range(len(panos)):
#for i in np.arange(0,len(panos)):
for i in [0]:
	pano_id = panos[i].pano_id

	of = "image_%02d" % i

	image = get_panorama_tiles(pano_id=pano_id, multi_threaded=True, outfile=of)
	image_cropped = Image.fromarray(autocrop(np.array(image)).astype('uint8'), 'RGB')
	image_cropped.save("image_%02d.jpg" % i, "jpeg")

	print("%s.jpg" % of, list(panos[i].__dict__.values()))
